# FBSDE

Ji, Shaolin, Shige Peng, Ying Peng, and Xichuan Zhang. “Three Algorithms for Solving High-Dimensional Fully-Coupled FBSDEs through Deep Learning.” ArXiv:1907.05327 [Cs, Math], February 2, 2020. http://arxiv.org/abs/1907.05327.

In [1]:
import numpy as np
import tensorflow as tf
from keras.layers import Input, Dense, Lambda, Reshape, concatenate
from keras import Model, initializers
import matplotlib.pyplot as plt

2022-03-17 19:18:36.278795: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices("GPU")))

Num GPUs Available:  4


2022-03-17 19:19:01.385130: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-03-17 19:19:01.569975: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:03:00.0 name: NVIDIA A100-SXM4-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.59GiB deviceMemoryBandwidth: 1.41TiB/s
2022-03-17 19:19:01.571665: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 1 with properties: 
pciBusID: 0000:44:00.0 name: NVIDIA A100-SXM4-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.59GiB deviceMemoryBandwidth: 1.41TiB/s
2022-03-17 19:19:01.573331: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 2 with properties: 
pciBusID: 0000:84:00.0 name: NVIDIA A100-SXM4-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.59GiB deviceMemoryBandwidth: 1.41TiB/s
2

In [3]:
n_paths = 256
n_timesteps = 10
n_dimensions = 4
n_factors = 2
T = 1.
dt = T / n_timesteps
batch_size = 128
epochs = 100

In [4]:
# convention: x = (s, alpha, q, c)

In [5]:
eta = 1.
lp = 1.
lm = 1.
k = 0.01
sigma = 1.
zeta = 1.
phi = 1.
psi = 1.

In [6]:
def b(t, x, y, z):
    return [
        x[1],
        -eta * x[0],
        lp * tf.exp(-1 + k * y[2] / y[3] + x[0] * k) - lm * tf.exp(-1 - k * y[2] / y[3] - x[0] * k),
        lp * (1./k - y[2] / y[3]) * tf.exp(-1 + k * y[2] / y[3] + x[0] * k) - lm * (-1./k - y[2] / y[3]) * tf.exp(-1. - k * y[2] / y[3] - x[0] * k),
    ]

def s(t, x, y, z):
    return [[sigma, 0], [0, zeta], [0, 0], [0, 0]]

def dH_dx(t, x, y, z):
    return [
        y[3] * lp * tf.exp(-1. + k * y[2] / y[3] + x[0] * k) - y[3] * lm * tf.exp(-1. - k * y[2] / y[3] - x[0] * k),
        y[0] - eta * y[1],
        -2. * phi * x[2],
        0.
    ]

def dg_dx(x):
    return [x[2], 0., x[0], x[3]]

In [7]:
paths = []

inputs_dW = Input(shape=(n_timesteps, n_factors))

x0 = tf.constant([[0., 0., 0., 0.]])
y0 = tf.Variable([[1., 1., 1., 1.]])

x = x0
y = y0

z = concatenate([x, y])
z = Dense(10, activation='relu', kernel_initializer=initializers.RandomNormal(stddev=1e-2))(z)
z = Dense(n_dimensions * n_factors, activation='relu', kernel_initializer=initializers.RandomNormal(stddev=1e-2))(z)
z = Reshape((n_dimensions, n_factors))(z)

paths += [[x, y, z]]

for i in range(n_timesteps):
            
    def dX(x, y, z, dw):
        
        def drift(arg):
            x, y, z = arg
            return tf.math.multiply(b(i*dt, x, y, z), dt)
        a0 = tf.vectorized_map(drift, (x, y, z))

        def noise(arg):
            x, y, z, dw = arg
            return tf.tensordot(s(i*dt, x, y, z), dw[i], [[1], [0]])
        a1 = tf.vectorized_map(noise, (x, y, z, dw))
        
        return a0 + a1

    def dY(x, y, z, dw):
        
        def drift(arg):
            x, y, z = arg
            return tf.math.multiply(dH_dx(i*dt, x, y, z), -dt)
        a0 = tf.vectorized_map(drift, (x, y, z))

        def noise(arg):
            x, y, z, dw = arg
            return tf.tensordot(z, dw[i], [[1], [0]])
        a1 = tf.vectorized_map(noise, (x, y, z, dw))
        
        return a0 + a1
    
    x, y = (
        Lambda(lambda r: r[0] + dX(r[0], r[1], r[2], r[3]))([x, y, z, inputs_dW]),
        Lambda(lambda r: r[1] + dY(r[0], r[1], r[2], r[3]))([x, y, z, inputs_dW]),
    )
    
    # we don't train z for the last time step; keep for consistency
    z = concatenate([x, y])
    z = Dense(10, activation='relu')(z)
    z = Dense(n_dimensions * n_factors, activation='relu')(z)
    z = Reshape((n_dimensions, n_factors))(z)

    paths += [[x, y, z]]
    
outputs_loss = Lambda(lambda r: r[1] - tf.transpose(tf.vectorized_map(dg_dx, r[0])))([x, y])
model_loss = Model(inputs_dW, outputs_loss)
model_loss.compile(loss='mse', optimizer='adam')

# (n_sample, n_timestep, x/y/z_k, n_dimension)
# skips the first time step
outputs_paths = tf.stack([tf.stack([p[0] for p in paths[1:]], axis=1), tf.stack([p[1] for p in paths[1:]], axis=1)] + [tf.stack([p[2][:, :, i] for p in paths[1:]], axis=1) for i in range(n_factors)], axis=2)
model_paths = Model(inputs_dW, outputs_paths)

2022-03-17 19:19:03.147272: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:03:00.0 name: NVIDIA A100-SXM4-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.59GiB deviceMemoryBandwidth: 1.41TiB/s
2022-03-17 19:19:03.148950: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 1 with properties: 
pciBusID: 0000:44:00.0 name: NVIDIA A100-SXM4-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.59GiB deviceMemoryBandwidth: 1.41TiB/s
2022-03-17 19:19:03.150596: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 2 with properties: 
pciBusID: 0000:84:00.0 name: NVIDIA A100-SXM4-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.59GiB deviceMemoryBandwidth: 1.41TiB/s
2022-03-17 19:19:03.152223: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 3 with properties: 
pciBusID: 0000:c4:00.0 n

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
The following Variables were used a Lambda layer's call (lambda), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=(1, 4) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (lambda_1), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=(1, 4) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.


# Training

In [ ]:
dW = tf.sqrt(dt) * tf.random.normal((n_paths, n_timesteps, n_factors))
target = tf.zeros((n_paths, 4))
model_loss.fit(dW, target, batch_size=batch_size, epochs=epochs)

# Display paths and loss

In [ ]:
model_loss(dW).numpy()

In [10]:
paths = model_paths(dW).numpy()

In [ ]:
np.set_printoptions(edgeitems=30, linewidth=100000, formatter=dict(float=lambda x: "%9.4g" % x))
tf.transpose(paths[3, :, :, :], (1, 2, 0)).numpy()